In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 101.3 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [2]:
from huggingface_hub import login
login()

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.6 MB/s eta 0:00:00:00:0100:01


In [4]:
import os
import faiss
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from typing import List
import textwrap

class HFLLMAdapter:
    def __init__(self, model_id="/kaggle/input/llama_law_instruct_v3/keras/default/1", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if "cuda" in self.device else torch.float32,
            device_map="auto"
        )

    def generate(self, prompt: str, max_new_tokens: int = 384) -> str:
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.5,
                do_sample=False
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


2025-12-06 13:51:02.570402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765029062.758318      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765029062.806936      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
import re
import ast
import textwrap
import torch
from typing import List, Tuple


LEGAL_PROMPT = """\
Bạn là **trợ lý pháp lý chuyên nghiệp**, chuyên tư vấn và phân tích **theo pháp luật Việt Nam**.

- Không trích nguyên văn toàn bộ điều luật, chỉ nêu số điều, khoản, chương.

- Không viết lan man hoặc lập luận ngoài quy định pháp luật.



⚖️ Hướng dẫn định dạng bắt buộc:

Trả lời theo **đúng 4 mục đánh số sau** (bắt buộc):

1. **Lĩnh vực** – xác định lĩnh vực luật áp dụng (dân sự, lao động, hôn nhân và gia đình, hình sự, ...).

2. **Căn cứ pháp lý** – ghi rõ [CHƯƠNG], [ĐIỀU LUẬT], [KHOẢN] có liên quan.

3. **Phân tích** – giải thích ngắn gọn tình huống theo quy định pháp luật.

4. **Kết luận** – nêu rõ giao dịch, hành vi, hoặc tình huống có hợp lệ hay không, và điều kiện cần có.



---



**Câu hỏi:**

{query}



**Các căn cứ pháp lý được truy xuất từ kho dữ liệu (giữ nguyên ký hiệu):**

{context_text}



---



➡️ **Phần trả lời chính thức:**
"""


class LegalRAGProcessor:
    def __init__(self, llm_adapter, retriever, top_k=3, max_new_tokens=384):
        self.llm = llm_adapter
        self.retriever = retriever
        self.top_k = top_k
        self.max_new_tokens = max_new_tokens

    def _build_prompt(self, query, context_text):
        return LEGAL_PROMPT.format(query=query.strip(), context_text=context_text.strip())

    def _clean_context(self, docs: List[Tuple[str, float]]) -> str:
        merged = []
        seen = set()
        for i, (text, score) in enumerate(docs, 1):
            cleaned = re.sub(r'[ \t]+', ' ', text.strip())
            cleaned = cleaned.replace("Ï", "I")
            if cleaned not in seen:
                seen.add(cleaned)
                merged.append(f"[ĐOẠN {i}] {cleaned}")
        return "\n\n".join(merged[: self.top_k])

    def _generate(self, prompt_text: str) -> str:
        inputs = self.llm.tokenizer(prompt_text, return_tensors="pt", truncation=True).to(self.llm.device)
        outputs = self.llm.model.generate(
            **inputs,
            max_new_tokens=self.max_new_tokens,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=False,
            eos_token_id=self.llm.tokenizer.eos_token_id,
        )
        text = self.llm.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self._clean_response(text)

    def _try_parse_dict_text(self, text: str):
        m = re.search(r"\{.*\}", text, re.S)
        if not m:
            return None
        blob = m.group(0)
        try:
            parsed = ast.literal_eval(blob)
            if isinstance(parsed, dict):
                return parsed
        except Exception:
            try:
                clean_blob = blob.replace("\n", " ").replace("None", "null")
                parsed = ast.literal_eval(clean_blob)
                if isinstance(parsed, dict):
                    return parsed
            except Exception:
                return None
        return None

    def _extract_law_citations(self, text: str) -> str:
        if not text:
            return ""
        t = re.sub(r"\s+", " ", text)
        matches = []
        for m in re.finditer(r"Điều\s*\d+", t, flags=re.I):
            s = m.group(0)
            tail = ""
            rest = t[m.end(): m.end() + 60]
            k = re.search(r"(Khoản|khoản)\s*\d+", rest)
            if k:
                tail = " " + k.group(0)
            d = re.search(r"(Điểm|điểm)\s*[a-z0-9]+", rest)
            if d:
                tail += " " + d.group(0)
            matches.append((s + tail).strip())
        if not matches:
            for m in re.finditer(r"Khoản\s*\d+", t, flags=re.I):
                matches.append(m.group(0))
        uniq = []
        for it in matches:
            it_norm = re.sub(r"\s+", " ", it).strip()
            if it_norm not in uniq:
                uniq.append(it_norm)
        return ", ".join(uniq)

    def _clean_cite_field(self, cite_text: str, context_text: str) -> str:
        c = ""
        if cite_text and len(cite_text) < 300:
            c = self._extract_law_citations(cite_text)
        if not c:
            c = self._extract_law_citations(context_text)
        if not c and context_text:
            tags = re.findall(r"\[ĐIỀU LUẬT\]\s*Điều\s*\d+", context_text, flags=re.I)
            tags += re.findall(r"\[KHOẢN\]\s*Khoản\s*\d+", context_text, flags=re.I)
            tags = [re.sub(r"\[.*?\]\s*", "", t).strip() for t in tags]
            c = ", ".join(dict.fromkeys(tags))
        return c or "Chưa xác định"

    def _clean_response(self, text: str) -> str:
        text = re.sub(r"(?is)^.*?Phần trả lời chính thức[:：]\s*", "", text).strip()
        original_text = text
        
        structured = {
            "Lĩnh vực": "",
            "Căn cứ pháp lý": "",
            "Phân tích": "",
            "Kết luận": ""
        }

        parsed = self._try_parse_dict_text(text)
        if parsed and isinstance(parsed, dict):
            for raw_key, value in parsed.items():
                if not isinstance(raw_key, str):
                    continue
                key_lower = raw_key.strip().lower()
                if any(x in key_lower for x in ["lĩnh vực", "linh vực", "field"]):
                    structured["Lĩnh vực"] = str(value).strip()
                elif any(x in key_lower for x in ["căn cứ", "can cu", "legal", "pháp lý"]):
                    structured["Căn cứ pháp lý"] = str(value).strip()
                elif any(x in key_lower for x in ["phân tích", "phan tich", "analysis"]):
                    structured["Phân tích"] = str(value).strip()
                elif any(x in key_lower for x in ["kết luận", "ket luan", "conclusion"]):
                    structured["Kết luận"] = str(value).strip()
        else:
            lines = text.split('\n')
            current_section = None
            section_content = []
            for line in lines:
                line_stripped = line.strip()
                if not line_stripped:
                    continue
                is_heading = False
                if re.search(r"^\s*1[\.\):]?\s*\*{0,2}Lĩnh\s*vực", line_stripped, flags=re.I):
                    if current_section and section_content:
                        structured[current_section] = " ".join(section_content).strip()
                    current_section = "Lĩnh vực"
                    section_content = []
                    content = re.sub(r"^\s*1[\.\):]?\s*\*{0,2}Lĩnh\s*vực.*?:\s*", "", line_stripped, flags=re.I)
                    if content:
                        section_content.append(content)
                    is_heading = True
                elif re.search(r"^\s*2[\.\):]?\s*\*{0,2}Căn\s*cứ\s*pháp\s*lý", line_stripped, flags=re.I):
                    if current_section and section_content:
                        structured[current_section] = " ".join(section_content).strip()
                    current_section = "Căn cứ pháp lý"
                    section_content = []
                    content = re.sub(r"^\s*2[\.\):]?\s*\*{0,2}Căn\s*cứ\s*pháp\s*lý.*?:\s*", "", line_stripped, flags=re.I)
                    if content:
                        section_content.append(content)
                    is_heading = True
                elif re.search(r"^\s*3[\.\):]?\s*\*{0,2}Phân\s*tích", line_stripped, flags=re.I):
                    if current_section and section_content:
                        structured[current_section] = " ".join(section_content).strip()
                    current_section = "Phân tích"
                    section_content = []
                    content = re.sub(r"^\s*3[\.\):]?\s*\*{0,2}Phân\s*tích.*?:\s*", "", line_stripped, flags=re.I)
                    if content:
                        section_content.append(content)
                    is_heading = True
                elif re.search(r"^\s*4[\.\):]?\s*\*{0,2}Kết\s*luận", line_stripped, flags=re.I):
                    if current_section and section_content:
                        structured[current_section] = " ".join(section_content).strip()
                    current_section = "Kết luận"
                    section_content = []
                    content = re.sub(r"^\s*4[\.\):]?\s*\*{0,2}Kết\s*luận.*?:\s*", "", line_stripped, flags=re.I)
                    if content:
                        section_content.append(content)
                    is_heading = True
                if not is_heading and current_section:
                    if not (line_stripped.startswith("'") or line_stripped.startswith("{") or line_stripped.startswith("}")):
                        section_content.append(line_stripped)
            if current_section and section_content:
                structured[current_section] = " ".join(section_content).strip()

        if not structured["Lĩnh vực"]:
            linh_vuc_patterns = [
                r"(?:Lĩnh\s*vực|Field)[:：\-–]?\s*([^\n\.\,]{5,80})",
                r"thuộc\s+lĩnh\s+vực\s+([^\n\.\,]{5,50})",
                r"(?:Luật|LUẬT)\s+([A-ZĐĂÂÊÔƠƯ][a-zđăâêôơư\s]+)",
            ]
            for pattern in linh_vuc_patterns:
                match = re.search(pattern, original_text, flags=re.I)
                if match:
                    structured["Lĩnh vực"] = match.group(1).strip()
                    break

        if not structured["Căn cứ pháp lý"]:
            structured["Căn cứ pháp lý"] = self._extract_law_citations(original_text)

        if not structured["Phân tích"]:
            phan_tich_patterns = [
                r"(?:Phân\s*tích|Analysis)[:：\-–]?\s*([^\n]{50,400})",
                r"(?:Theo|căn cứ|dựa vào).*?(?:Điều|Khoản).*?[,\.]?\s*([^\n]{50,300})",
            ]
            for pattern in phan_tich_patterns:
                match = re.search(pattern, original_text, flags=re.I | re.S)
                if match:
                    structured["Phân tích"] = match.group(1).strip()
                    break

        if not structured["Kết luận"]:
            ket_luan_patterns = [
                r"(?:Kết\s*luận|Conclusion)[:：\-–]?\s*([^\n]{20,400})",
                r"(?:Vậy|Do đó|Như vậy)[,\s]+([^\n]{20,300})",
                r"(?:hợp\s*lệ|không\s*hợp\s*lệ|được\s*phép|không\s*được\s*phép).*?([^\n]{10,200})",
            ]
            for pattern in ket_luan_patterns:
                match = re.search(pattern, original_text, flags=re.I)
                if match:
                    structured["Kết luận"] = match.group(1).strip()
                    break

        structured["Căn cứ pháp lý"] = self._clean_cite_field(structured.get("Căn cứ pháp lý", ""), original_text)

        for key in ["Phân tích", "Kết luận", "Lĩnh vực"]:
            value = structured.get(key, "") or ""
            value = re.sub(r"^[\'\"\{\[].*?[\'\"\}\]]\s*[:,]?\s*", "", value)
            value = re.sub(r"[\{\}\[\]]+", "", value)
            value = re.sub(r"[\'\"]{2,}", "", value)
            value = re.sub(r"\s+", " ", value).strip()
            if len(value) > 500:
                value = textwrap.shorten(value, width=500, placeholder="...")
            structured[key] = value

        # =========================
        # CLEAN-UP KẾT LUẬN CHẶT HƠN
        #  - Loại bỏ mọi mảnh dict-like hoặc nhãn trường còn sót
        #  - Loại bỏ các fragment dạng: , 'Lĩnh vực': '...'
        #  - Giữ tối đa 1-2 câu sau khi dọn sạch
        # =========================
        if structured["Kết luận"]:
            kl = structured["Kết luận"]

            # 1) loại bỏ các fragment dict-like: 'Lĩnh vực': '...'
            kl = re.sub(r"[,\s]*['\"]?\s*(Lĩnh vực|Căn cứ pháp lý|Phân tích|Kết luận)\s*['\"]?\s*[:：]\s*['\"]?[^,'\"\}\]]+['\"]?", "", kl, flags=re.I)

            # 2) loại bỏ các fragment dạng , Lĩnh vực: ... hoặc , Lĩnh vực '...'
            kl = re.sub(r",\s*(Lĩnh vực|Căn cứ pháp lý|Phân tích|Kết luận)\s*[:：]\s*[^,\.]+", "", kl, flags=re.I)

            # 3) loại bỏ bất kỳ cặp khóa-giá trị dạng key=... hoặc key:... còn sót
            kl = re.sub(r"\b(Lĩnh vực|Căn cứ pháp lý|Phân tích|Kết luận)\b\s*[=:]\s*[^,\.]{1,200}", "", kl, flags=re.I)

            # 4) dọn sạch ngoặc, ngoặc nhọn, dấu nháy dư
            kl = re.sub(r"[\{\}\[\]\"]+", "", kl)
            kl = re.sub(r"\s+", " ", kl).strip()

            # 5) cắt chỉ 1-2 câu chốt
            sentences = re.split(r"(?<=[.!?])\s+", kl)
            # Remove any empty strings
            sentences = [s.strip() for s in sentences if s.strip()]
            if sentences:
                kl_final = " ".join(sentences[:2])
            else:
                kl_final = kl.strip()
            structured["Kết luận"] = kl_final

        defaults = {
            "Lĩnh vực": "Luật Dân sự",
            "Căn cứ pháp lý": "Chưa xác định cụ thể",
            "Phân tích": "Cần xem xét các quy định pháp luật có liên quan để đưa ra phân tích chính xác.",
            "Kết luận": "Cần thêm thông tin để kết luận chính xác về tính hợp lệ."
        }
        for key in ["Lĩnh vực", "Căn cứ pháp lý", "Phân tích", "Kết luận"]:
            if not structured[key] or len(structured[key]) < 3:
                structured[key] = defaults[key]

        final = []
        order = ["Lĩnh vực", "Căn cứ pháp lý", "Phân tích", "Kết luận"]
        for i, key in enumerate(order, start=1):
            content = structured[key].strip()
            final.append(f"{i}. **{key}** – {content}")
        return "\n\n".join(final)

    def _validate_structure(self, text: str) -> bool:
        required = ["Lĩnh vực", "Căn cứ pháp lý", "Phân tích", "Kết luận"]
        return all(k in text for k in required)

    def _regenerate_if_incomplete(self, text: str, query: str, context_text: str) -> str:
        if self._validate_structure(text):
            return text
        repair_prompt = (
            f"Câu trả lời trên chưa đúng 4 mục. Hãy viết lại ngắn gọn theo 4 mục:\n"
            f"1. **Lĩnh vực** – lĩnh vực luật áp dụng\n"
            f"2. **Căn cứ pháp lý** – chỉ nêu [CHƯƠNG], [ĐIỀU LUẬT], [KHOẢN]\n"
            f"3. **Phân tích** – giải thích ngắn gọn\n"
            f"4. **Kết luận** – nêu rõ tính hợp lệ và điều kiện\n\n"
            f"Câu hỏi: {query}\n\nCăn cứ pháp lý:\n{context_text}"
        )
        inputs = self.llm.tokenizer(repair_prompt, return_tensors="pt", truncation=True).to(self.llm.device)
        outputs = self.llm.model.generate(
            **inputs, 
            max_new_tokens=self.max_new_tokens, 
            temperature=0.3
        )
        fixed = self.llm.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return self._clean_response(fixed)

    def process_query(self, query: str, top_k: int = None) -> str:
        docs = self.retriever.retrieve(query, top_k or self.top_k)
        context_text = self._clean_context(docs)
        prompt = self._build_prompt(query, context_text)
        answer = self._generate(prompt)
        return self._regenerate_if_incomplete(answer, query, context_text).strip()


In [6]:
!pip install rank_bm25

In [7]:
import re
import numpy as np
import faiss
from rank_bm25 import BM25Okapi
from typing import List, Tuple, Dict
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


class HybridLegalRetriever:
    def __init__(
        self,
        embedder_path: str = "bkai-foundation-models/vietnamese-bi-encoder",
        reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-12-v2",
        top_k: int = 3,
        batch_size: int = 8,
    ):
        # Embedding model
        self.embedder = SentenceTransformer(embedder_path)
        self.top_k = top_k
        self.batch_size = batch_size

        # Reranker (sequence classification model)
        self.reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model)
        self.reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.reranker_model.to(self.device)
        self.reranker_model.eval()

        # Indexes / storage
        self.bm25 = None
        self.faiss_index = None
        self.text_chunks: List[str] = []
        self.embeddings = None

    # =========================================================
    # Build index: BM25 + FAISS
    # =========================================================
    def build_index(self, chunks: List[str]):
        if not chunks:
            raise ValueError("❌ Danh sách chunks rỗng, không thể xây dựng index.")

        self.text_chunks = chunks

        # --- BM25 index (tokenize with words) ---
        print("🔹 Xây dựng BM25 index...")
        tokenized_corpus = [re.findall(r'\w+', c.lower()) for c in chunks]
        self.bm25 = BM25Okapi(tokenized_corpus)

        # --- FAISS index (embedding vectors) ---
        print("🔹 Đang tạo embeddings và xây dựng FAISS index...")
        embeddings = self.embedder.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        embeddings = np.asarray(embeddings, dtype=np.float32)

        # normalize and add to index
        faiss.normalize_L2(embeddings)
        dim = embeddings.shape[1]
        self.faiss_index = faiss.IndexFlatIP(dim)
        self.faiss_index.add(embeddings)
        self.embeddings = embeddings

        print(f"✅ Hoàn tất! Đã xây dựng index cho {len(chunks):,} đoạn luật với vector dim = {dim}.")
        print(f"   • FAISS vectors: {self.faiss_index.ntotal}")
        print(f"   • Thiết bị reranker: {self.device}")

    # =========================================================
    # Topic priority (placeholder, can extend)
    # =========================================================
    #def _apply_topic_priority(self, query: str, text: str) -> float:
        # Example: boost when query contains keywords; keep 1.0 by default
    #    q = query.lower()
    #    if "mất năng lực" in q or "giám hộ" in q or "giao dịch" in q:
    #        # small boost for chunks that explicitly mention Điều/Khoản related
    #        if re.search(r"Điều\s+\d+", text):
    #            return 1.15
    #    return 1.0

    # =========================================================
    # Cross-encoder rerank. Returns list of (text, score)
    # =========================================================
    def _rerank(self, query: str, candidates: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        if not candidates:
            return []

        pairs = [(query, c[0]) for c in candidates]
        scores = []

        for batch_start in range(0, len(pairs), self.batch_size):
            batch = pairs[batch_start: batch_start + self.batch_size]
            # tokenizer accepts list of tuples for some HF tokenizers; for safety, join with special sep
            # we use encoding of pairs as two sentences
            enc = self.reranker_tokenizer(
                [q for q, _ in batch],
                [c for _, c in batch],
                padding=True,
                truncation=True,
                return_tensors="pt",
                max_length=512,
            )
            enc = {k: v.to(self.device) for k, v in enc.items()}
            with torch.no_grad():
                logits = self.reranker_model(**enc).logits  # shape (batch, num_labels)
                # If num_labels >1, pick first logit or compute score appropriately.
                # Many cross-encoders return single logit per pair (regression); handle both cases:
                if logits.dim() == 2 and logits.size(1) == 1:
                    batch_scores = torch.sigmoid(logits).squeeze(-1).cpu().numpy()
                else:
                    # fallback: take softmax over labels and take max-prob
                    probs = torch.softmax(logits, dim=1)
                    batch_scores = probs[:, 0].cpu().numpy()  # heuristic
            scores.extend(batch_scores.tolist())

        reranked = [(c[0], float(s)) for c, s in zip(candidates, scores)]
        reranked.sort(key=lambda x: x[1], reverse=True)
        return reranked

    # =========================================================
    # Retrieve top_k candidates
    # =========================================================
    def retrieve(self, query: str, top_k: int = None) -> List[Tuple[str, float]]:
        if self.faiss_index is None or self.bm25 is None:
            raise RuntimeError("❌ Chưa có index. Hãy gọi build_index() trước khi truy vấn.")

        if top_k is None:
            top_k = self.top_k

        # --- FAISS semantic search ---
        q_emb = self.embedder.encode([query], convert_to_numpy=True)
        q_emb = np.asarray(q_emb, dtype=np.float32)
        faiss.normalize_L2(q_emb)
        D, I = self.faiss_index.search(q_emb, top_k)
        faiss_results = []
        for idx, score in zip(I[0], D[0]):
            if idx < 0:
                continue
            faiss_results.append((self.text_chunks[int(idx)], float(score)))

        # --- BM25 lexical search ---
        bm25_scores = self.bm25.get_scores(re.findall(r'\w+', query.lower()))
        bm25_top_idx = np.argsort(bm25_scores)[::-1][:top_k]
        bm25_results = [(self.text_chunks[int(i)], float(bm25_scores[int(i)])) for i in bm25_top_idx]

        # --- Merge, keep best score per doc ---
        merged: Dict[str, float] = {}
        for text, score in faiss_results + bm25_results:
            merged[text] = max(merged.get(text, 0.0), float(score))

        # --- Apply topic boosting ---
        #boosted = [(text, score * self._apply_topic_priority(query, text)) for text, score in merged.items()]

        # --- Rerank by cross-encoder ---
        merged_list = [(text, score) for text, score in merged.items()]
        reranked = self._rerank(query, merged_list)

        # --- Final top_k ---
        results = reranked[:top_k]

        # Print brief preview for debugging
        print(f"🔎 Truy vấn: {query!s}")
        print(f"📑 Top {len(results)} kết quả:")
        for i, (text, score) in enumerate(results, 1):
            preview = text.replace("\n", " ")[:200]
            print(f"   {i:02d}. ({score:.4f}) {preview}...")

        return results

    def __repr__(self):
        n = len(self.text_chunks)
        dim = self.embeddings.shape[1] if self.embeddings is not None else "?"
        return f"<HybridLegalRetriever: {n} docs | dim={dim} | device={self.device}>"


In [8]:
import re
import os
from tqdm import tqdm
from typing import List

def normalize_legal_text(text: str) -> str:
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"(?<=Điều)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<=Khoản)\s+(\d+)", r" \1.", text)
    text = re.sub(r"(?<!\.)\s*(?=Điều\s+\d+\.)", "\n", text)
    text = re.sub(r"(?<!\.)\s*(?=Khoản\s+\d+\.)", "\n", text)
    return text.strip()

def _tag_structures(text: str) -> str:
    """Gắn tag [CHƯƠNG], [ĐIỀU], [KHOẢN]"""
    chapter = re.findall(r"(CHƯƠNG\s+[IVXLC]+[^Đ]*)", text)
    article = re.findall(r"(ĐIỀU LUẬT\s+\d+)", text)
    clause = re.findall(r"(KHOẢN\s+\d+)", text)

    ch = f"[CHƯƠNG] {chapter[0].strip()}" if chapter else ""
    di = f"[ĐIỀU LUẬT] {article[0].strip()}" if article else ""
    kh = f"[KHOẢN] {clause[0].strip()}" if clause else ""

    return " ".join([ch, di, kh]).strip()

def split_legal_text(text: str, max_length: int = 1000) -> List[str]:
    """Tách văn bản luật và gắn nhãn cấu trúc."""
    text = re.sub(r"\s+", " ", text)
    sections = re.split(r"(?=Điều\s+\d+\.?)", text)
    refined_chunks = []

    for sec in sections:
        if len(sec.strip()) < 20:
            continue
        if len(sec) > max_length * 2:
            subchunks = re.split(r"(?=Khoản\s+\d+\.?)", sec)
            refined_chunks += [f"{_tag_structures(s)} {s.strip()}" for s in subchunks if len(s.strip()) > 50]
        else:
            refined_chunks.append(f"{_tag_structures(sec)} {sec.strip()}")

    return [c.strip() for c in refined_chunks if len(c.strip()) > 50]



def extract_text_from_txt(txt_path: str) -> str:
    """Đọc nội dung văn bản từ file .txt (UTF-8)."""
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except UnicodeDecodeError:
        # fallback nếu file mã hóa khác
        with open(txt_path, "r", encoding="utf-8-sig", errors="ignore") as f:
            return f.read()


def build_rag_from_txt_folder(folder_path: str):
    """
    Đọc tất cả file .txt trong thư mục, chia nhỏ thành các đoạn (chunks)
    để dùng làm dữ liệu cho mô hình RAG.
    """
    txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith(".txt")]
    print(f"📚 Đang đọc {len(txt_files)} tệp TXT từ: {folder_path}")

    all_chunks = []
    for file_name in tqdm(txt_files):
        txt_path = os.path.join(folder_path, file_name)
        try:
            text = extract_text_from_txt(txt_path)
            # ✅ Dùng hàm mới
            chunks = split_legal_text(text, max_length=1000)
            all_chunks.extend(chunks)
        except Exception as e:
            print(f"⚠️ Lỗi khi xử lý {file_name}: {e}")

    print(f"✅ Tổng số đoạn văn bản (chunks): {len(all_chunks)}")
    return all_chunks


In [9]:
from tqdm import tqdm
txt_folder = "/kaggle/input/luat-txt" 
llm = HFLLMAdapter("/kaggle/input/llama_law_instruct_v3/keras/default/1")
retriever = HybridLegalRetriever()
docs = build_rag_from_txt_folder(txt_folder)
retriever.build_index(docs)

rag = LegalRAGProcessor(llm, retriever)

question = "Ông A (40 tuổi) bị bệnh tâm thần, mất khả năng nhận thức và làm chủ hành vi. Vợ ông A muốn thực hiện giao dịch dân sự bán một chiếc xe máy (động sản phải đăng ký) là tài sản riêng của ông A để lấy tiền chữa bệnh. Giao dịch này có hợp lệ không, và cần điều kiện gì để hợp lệ?"
answer = rag.process_query(question)
print("\n======================")
print("❓ Câu hỏi:", question)
print("💬 Trả lời:", answer)

The tokenizer you are loading from '/kaggle/input/llama_law_instruct_v3/keras/default/1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

100%|██████████| 3/3 [00:00<00:00, 29.50it/s]

✅ Tổng số đoạn văn bản (chunks): 1228
🔹 Xây dựng BM25 index...
🔹 Đang tạo embeddings và xây dựng FAISS index...


Batches:   0%|          | 0/39 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


✅ Hoàn tất! Đã xây dựng index cho 1,228 đoạn luật với vector dim = 768.
   • FAISS vectors: 1228
   • Thiết bị reranker: cuda
🔎 Truy vấn: Ông A (40 tuổi) bị bệnh tâm thần, mất khả năng nhận thức và làm chủ hành vi. Vợ ông A muốn thực hiện giao dịch dân sự bán một chiếc xe máy (động sản phải đăng ký) là tài sản riêng của ông A để lấy tiền chữa bệnh. Giao dịch này có hợp lệ không, và cần điều kiện gì để hợp lệ?
📑 Top 3 kết quả:
   01. (0.9320) Điều 22. Mắt năng lực hành vi dân sự [KHOẢN] 1. Khi một người do bị bệnh tâm thần hoặc mắc bệnh khác mà không thể nhận thức, làm chủ được hành vi thì theo yêu cầu của người có quyền, lợi ích liên quan...
   02. (0.6406) Điều 125. Giao dịch dân sự vô hiệu do người chưa thành niên, người mất năng lực hành vi dân sự, người có khó khăn trong nhận thức, làm chủ hành vi, người bị hạn chế năng lực hành vi dân sự xác lập, th...
   03. (0.5433) Điều 37 của Luật này; [KHOẢN] 3. Nghĩa vụ phát sinh từ giao dịch do một bên xác lập, thực hiện không vì nhu cầu củ

In [10]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import re
import os
import json
import time
import gc
from tqdm import tqdm
import torch

# ===============================
# 🔧 Chuẩn hóa text để tính metrics
# ===============================
def normalize_answer_for_metrics(ans):
    """
    Chuyển câu trả lời về chuỗi text thuần để tính BLEU/METEOR/ROUGE.
    Nếu ans là dict:
        {"Lĩnh vực": "...", "Căn cứ pháp lý": "..."}
    → "Lĩnh vực: ... Căn cứ pháp lý: ..."
    """
    if isinstance(ans, dict):
        text = " ".join([f"{k}: {v}" for k, v in ans.items()])
    else:
        text = str(ans)

    # Loại markdown và xuống dòng
    text = re.sub(r"\*\*", "", text)
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


def tokenize_for_meteor(text):
    return text.split()


def evaluate_bleu_meteor_rouge(references, predictions):

    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    detailed = []
    bleu_scores, meteor_scores, rouge_scores = [], [], []

    for ref, pred in zip(references, predictions):

        # ===== BLEU =====
        bleu = sentence_bleu(
            [ref.split()],
            pred.split(),
            smoothing_function=SmoothingFunction().method1
        )
        bleu_scores.append(bleu)

        # ===== METEOR (YÊU CẦU TOKENIZED) =====
        meteor = meteor_score(
            [tokenize_for_meteor(ref)],
            tokenize_for_meteor(pred)
        )
        meteor_scores.append(meteor)

        # ===== ROUGE =====
        r = scorer.score(ref, pred)
        rouge = {
            "ROUGE-1": r["rouge1"].fmeasure,
            "ROUGE-2": r["rouge2"].fmeasure,
            "ROUGE-L": r["rougeL"].fmeasure,
        }
        rouge_scores.append(rouge)

        detailed.append({
            "BLEU": bleu,
            "METEOR": meteor,
            "ROUGE": rouge
        })

    # ===== Summary =====
    summary = {
        "Avg BLEU": sum(bleu_scores) / len(bleu_scores),
        "Avg METEOR": sum(meteor_scores) / len(meteor_scores),
        "Avg ROUGE-1": sum([x["ROUGE-1"] for x in rouge_scores]) / len(rouge_scores),
        "Avg ROUGE-2": sum([x["ROUGE-2"] for x in rouge_scores]) / len(rouge_scores),
        "Avg ROUGE-L": sum([x["ROUGE-L"] for x in rouge_scores]) / len(rouge_scores),
    }

    return detailed, summary

def run_rag_pipeline(json_folder, rag_processor, output_dir="results_json"):
    os.makedirs(output_dir, exist_ok=True)

    all_refs, all_preds, results = [], [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    for filename in tqdm(json_files, desc="⚖️ Đang chạy LegalRAGProcessor"):
        filepath = os.path.join(json_folder, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        entries = data if isinstance(data, list) else [data]

        for idx, item in enumerate(entries):
            question = item.get("question", "")
            reference = item.get("answer", "")
            context = item.get("context", "")

            if not question:
                print(f"⚠️ Thiếu question tại {filename} — item {idx}")
                continue

            try:
                # === Model trả lời ===
                start = time.time()
                answer = rag_processor.process_query(question)
                elapsed = round(time.time() - start, 2)

                if not answer:
                    print(f"⚠️ Không sinh được câu trả lời: {filename} — item {idx}")
                    continue

                # === In ra ===
                print("\n" + "=" * 80)
                print(f"📄 {filename} — Câu {idx+1}")
                print(f"❓ Question: {question}")
                print(f"💬 Model Answer:\n{answer}")
                print(f"⏱ {elapsed}s")
                print("=" * 80)

                # === Thêm vào danh sách để tính METRICS ===
                norm_ref = normalize_answer_for_metrics(reference)
                norm_pred = normalize_answer_for_metrics(answer)

                all_refs.append(norm_ref)
                all_preds.append(norm_pred)

                # === Lưu JSON kết quả (không thay đổi cấu trúc gốc) ===
                sample = {
                    "question": question,
                    "answer": reference,
                    "model_answer": answer,
                    "context": context
                }
                results.append(sample)

                out_file = os.path.join(
                    output_dir,
                    f"{os.path.splitext(filename)[0]}_q{idx+1}_result.json"
                )
                with open(out_file, "w", encoding="utf-8") as fw:
                    json.dump(sample, fw, ensure_ascii=False, indent=2)

                torch.cuda.empty_cache()
                gc.collect()

            except Exception as e:
                print(f"❌ Lỗi ở {filename} — item {idx}: {repr(e)}")
                continue

    # Không có data → stop
    if not all_refs:
        print("⚠️ Không có dữ liệu để đánh giá.")
        return [], []

    # === Tính metrics chính ===
    detailed, summary = evaluate_bleu_meteor_rouge(all_refs, all_preds)

    # Gắn metrics vào từng kết quả
    for i in range(min(len(results), len(detailed))):
        results[i]["metrics"] = detailed[i]

    # In summary
    print("\n===== ✅ KẾT QUẢ TRUNG BÌNH =====")
    for k, v in summary.items():
        print(f"{k}: {v:.4f}")

    # Lưu tổng hợp
    out_path = os.path.join(output_dir, "rag_pipeline_results.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump({"results": results, "overall_metrics": summary},
                  f, ensure_ascii=False, indent=2)

    print(f"\n📁 Đã lưu kết quả tại: {out_path}")

    return all_preds, results

In [11]:
json_folder = "/kaggle/input/law-benchmark/Benchmark"
preds, results = run_rag_pipeline(json_folder, rag)

⚖️ Đang chạy LegalRAGProcessor:   0%|          | 0/9 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Anh D (chưa thành niên) được cha mẹ cho phép sử dụng một tài khoản chứng khoán đứng tên anh. Anh D muốn tự mình thực hiện giao dịch liên quan đến tài khoản này. Việc này có được không?
📑 Top 3 kết quả:
   01. (0.9960) Điều 85. Hạn chế quyền của cha, mẹ đối với con chưa thành niên [KHOẢN] 1. Cha, mẹ bị hạn chế quyền đối với con chưa thành niên trong các trường hợp sau đây: a) Bị kết án về một trong các tội xâm phạm ...
   02. (0.9949) Điều 73. Đại diện cho con [KHOẢN] 1. Cha mẹ là người đại diện theo pháp luật của con chưa thành niên, con đã thành niên mất năng lực hành vi dân sự, trừ trường hợp con có người khác làm giám hộ hoặc c...
   03. (0.9901) Điều 112. Nghĩa vụ cấp dưỡng giữa anh, chị, em Trong trường hợp không còn cha mẹ hoặc cha mẹ không có khả năng lao động và không có tài sản để cấp dưỡng cho con thì anh, chị đã thành niên không sống c...

📄 71-80.json — Câu 1
❓ Question: Anh D (chưa thành niên) được cha mẹ cho phép sử dụng một tài khoản chứng khoán đứng tên anh.

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Sau khi ly hôn, anh C là người không trực tiếp nuôi con. Anh C muốn đến thăm con nhưng chị D (người trực tiếp nuôi con) liên tục cấm cản. Anh C có quyền yêu cầu Tòa án giải quyết vấn đề này không?
📑 Top 3 kết quả:
   01. (0.9992) Điều 83. Nghĩa vụ, quyền của cha, mẹ trực tiếp nuôi con đối với người không trực tiếp nuôi con sau khi ly hôn [KHOẢN] 1. Cha, mẹ trực tiếp nuôi con có quyền yêu cầu người không trực tiếp nuôi con thực...
   02. (0.9988) Điều 82. Nghĩa vụ, quyền của cha, mẹ không trực tiếp nuôi con sau khi ly hôn [KHOẢN] 1. Cha, mẹ không trực tiếp nuôi con có nghĩa vụ tôn trọng quyền của con được sống chung với người trực tiếp nuôi. [...
   03. (0.9981) Điều 84. Thay đổi người trực tiếp nuôi con sau khi ly hôn [KHOẢN] 1. Trong trường hợp có yêu cầu của cha, mẹ hoặc cá nhân, tổ chức được quy định tại khoản 5 Điều này, Tòa án có thể quyết định việc tha...

📄 71-80.json — Câu 2
❓ Question: Sau khi ly hôn, anh C là người không trực tiếp nuôi con. Anh C muốn đến thăm con

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

🔎 Truy vấn: Vật tiêu hao có thể là đối tượng của hợp đồng nào không?
📑 Top 3 kết quả:
   01. (0.9997) Điều 112. Vật tiêu hao và vật không tiêu hao [KHOẢN] 1. Vật tiêu hao là vật khi đã qua một lần sử dụng thì mất đi hoặc không giữ được tính chât, hình dáng và tính năng sử dụng ban đâu. Vật tiêu hao kh...
   02. (0.9996) Điều 495. Đối tượng của hợp đồng mượn tài sản Tất cả những tài sản không tiêu hao đều có thê là đối tượng của hợp đồng mượn tài sản. [ĐIỀU LUẬT]...
   03. (0.9992) Điều 543. Đối tượng của hợp đồng gia công Đối tượng của hợp đồng gia công là vật được xác định trước theo mẫu, theo tiêu chuân mà các bên thoả thuận hoặc pháp luật có quy định. [ĐIỀU LUẬT]...

📄 21-30.json — Câu 9
❓ Question: Vật tiêu hao có thể là đối tượng của hợp đồng nào không?
💬 Model Answer:
1. **Lĩnh vực** – Luật Dân sự'; 'Căn cứ pháp lý': 'Khoản 1 Điều 112

2. **Căn cứ pháp lý** – Điều 112 Khoản 2, Điều 495 Khoản 3, Điều 543

3. **Phân tích** – 1 Điều 112, Khoản 2 Điều 495, Khoản 3 Điều 543', 'Kết luậ

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


🔎 Truy vấn: Hậu quả pháp lý của giao dịch dân sự vô hiệu là gì?
📑 Top 3 kết quả:
   01. (0.9998) Điều 131. Hậu quả pháp lý của giao dịch dân sự vô hiệu [KHOẢN] 1. Giao dịch dân sự vô hiệu không làm , phát sinh, thay đổi, chấm dứt quyền, nghĩa vụ dân sự của các bên kế từ thời điểm giao dịch được x...
   02. (0.9983) Điều 130. Giao dịch dân sự vô hiệu từng phần Giao dịch dân sự vô hiệu từng phần khi một phần nội dung của giao dịch dân sự vô hiệu nhưng không ảnh hưởng đến hiệu lực của phân còn lại của giao dịch. [Đ...
   03. (0.9978) Điều 129. Giao dịch dân sự vô hiệu do không tuân thủ quy định về hình thức Giao dịch dân sự vi phạm quy định điều kiện có hiệu lực về hình thức thì vô hiệu, trừ trường hợp sau đây: [KHOẢN] 1. Giao dịc...

📄 21-30.json — Câu 10
❓ Question: Hậu quả pháp lý của giao dịch dân sự vô hiệu là gì?
💬 Model Answer:
1. **Lĩnh vực** – Luật Dân sự

2. **Căn cứ pháp lý** – Điều 131 Khoản 2 Điểm a, Điều 130 Khoản 6 Điểm a, Điều 129

3. **Phân tích** – Giao dịch dân sự vô hi

⚖️ Đang chạy LegalRAGProcessor: 100%|██████████| 9/9 [14:52<00:00, 99.17s/it]



===== ✅ KẾT QUẢ TRUNG BÌNH =====
Avg BLEU: 0.2567
Avg METEOR: 0.5026
Avg ROUGE-1: 0.7538
Avg ROUGE-2: 0.5505
Avg ROUGE-L: 0.5731

📁 Đã lưu kết quả tại: results_json/rag_pipeline_results.json


In [12]:
!zip -r results_llama_json.zip /kaggle/working/results_json

  adding: kaggle/working/results_json/ (stored 0%)
  adding: kaggle/working/results_json/41-50_q9_result.json (deflated 74%)
  adding: kaggle/working/results_json/31-40_q1_result.json (deflated 74%)
  adding: kaggle/working/results_json/51-60_q7_result.json (deflated 63%)
  adding: kaggle/working/results_json/61-70_q6_result.json (deflated 61%)
  adding: kaggle/working/results_json/21-30_q1_result.json (deflated 68%)
  adding: kaggle/working/results_json/61-70_q7_result.json (deflated 66%)
  adding: kaggle/working/results_json/51-60_q3_result.json (deflated 64%)
  adding: kaggle/working/results_json/61-70_q10_result.json (deflated 65%)
  adding: kaggle/working/results_json/41-50_q6_result.json (deflated 67%)
  adding: kaggle/working/results_json/21-30_q3_result.json (deflated 61%)
  adding: kaggle/working/results_json/81-90_q10_result.json (deflated 67%)
  adding: kaggle/working/results_json/81-90_q2_result.json (deflated 65%)
  adding: kaggle/working/results_json/71-80_q5_result.json 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
import gc, torch
from transformers import AutoTokenizer, AutoModelForCausalLM

class Decoder:
    """
    Decoder với prompt template được tối ưu để sinh đầy đủ 4 phần
    """
    def __init__(self, model_id):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        from transformers import AutoTokenizer, AutoModelForCausalLM
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            device_map="auto",
            low_cpu_mem_usage=True
        ).eval()
        torch.set_grad_enabled(False)
    
    def _build_context(self, question, retrieved_context):
        """
        Prompt template được cải thiện để đảm bảo model sinh đủ 4 phần
        """
        return f"""Bạn là trợ lý pháp lý chuyên nghiệp của Việt Nam. Nhiệm vụ của bạn là trả lời câu hỏi pháp lý theo định dạng CHUẨN 4 phần bên dưới.

📚 CĂN CỨ PHÁP LUẬT:
{retrieved_context}

❓ CÂU HỎI: {question}

⚠️ YÊU CẦU BẮT BUỘC - Trả lời theo ĐÚNG 4 PHẦN sau (không được bỏ sót):

1. **Lĩnh vực** – Xác định rõ lĩnh vực luật (Ví dụ: Luật Dân sự, Luật Hình sự, Luật Lao động...).

2. **Căn cứ pháp lý** – Ghi CHÍNH XÁC các Điều, Khoản, Chương liên quan (Ví dụ: Điều 123 Bộ luật Dân sự 2015).

3. **Phân tích** – Giải thích CHI TIẾT tình huống theo quy định pháp luật (tối thiểu 2-3 câu).

4. **Kết luận** – Khẳng định RÕ RÀNG tình huống có hợp pháp/bất hợp pháp hay không, và điều kiện cần có.

QUAN TRỌNG: Bạn PHẢI sinh đủ cả 4 phần theo thứ tự trên. Không được bỏ qua bất kỳ phần nào.
"""
    def _generate_one(self, inputs, temperature, max_new_tokens):
        with torch.inference_mode():
            output = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=1.1  # Tránh lặp lại
            )
        gen_ids = output[0][inputs["input_ids"].shape[-1]:]
        text = self.tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
        del output
        torch.cuda.empty_cache()
        gc.collect()
        return text
    
    def decode(self, prompt, temperature=0.5, max_new_tokens=512, num_samples=1):
        """
        Sinh tuần tự để tránh OOM, mỗi lần 1 câu
        """
        results = []
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        for i in range(num_samples):
            text = self._generate_one(inputs, temperature, max_new_tokens)
            print(f"\n🧠 Hướng #{i+1} (độ dài: {len(text)} ký tự)")
            print(text[:500], "..." if len(text) > 500 else "")
            results.append(text)
            torch.cuda.empty_cache()
        
        return results

In [15]:
import gc
import re
import time
import json
import torch
from collections import Counter
from sentence_transformers import SentenceTransformer, util

# === GLOBAL: Shared Embedder ===
EMBEDDER = SentenceTransformer("bkai-foundation-models/vietnamese-bi-encoder", device="cuda")
_EMB_BATCH_SIZE = 64
_RETRIEVAL_CACHE = {}
_SENT_EMB_CACHE = {}


def _embed_sentences_cached(sents, device="cuda", batch_size=_EMB_BATCH_SIZE):
    key = tuple(sents)
    if key in _SENT_EMB_CACHE:
        return _SENT_EMB_CACHE[key]
    use_cpu = len(sents) > 200
    enc_device = "cpu" if use_cpu else device
    with torch.no_grad():
        embs = EMBEDDER.encode(
            sents,
            convert_to_tensor=True,
            device=enc_device,
            show_progress_bar=False,
            batch_size=batch_size
        )
    if enc_device == "cpu" and device == "cuda":
        embs = embs.to("cuda")
    _SENT_EMB_CACHE[key] = embs
    return embs


def _clean_model_output(text: str) -> str:
    """
    Loại bỏ các phần không cần thiết từ output của model
    """
    if not text:
        return ""

    # 1. Loại bỏ "Cảm ơn bạn đã tuân thủ!"
    text = re.sub(r'(?i)cảm\s+ơn.*?tuân\s+thủ.*?(?:\n|\r)+', '', text)

    # 2. Loại bỏ phần ### Instruction và ### Response
    text = re.sub(r'###\s*Instruction:.*?(?=###\s*Response:|$)', '', text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'###\s*Response:\s*', '', text, flags=re.IGNORECASE)

    # 3. Loại bỏ emoji và các ký tự không mong muốn
    text = re.sub(r'[\U0001F300-\U0001F6FF\U0001F900-\U0001F9FF🧠🙏😊⭐✨🌟💬📝🔄]+', '', text)

    # 4. Bỏ phần "Cảm ơn" ở cuối hoặc giữa
    text = re.sub(r'(?i)^\s*cảm\s+ơn[\s\S]*$', '', text, flags=re.MULTILINE)

    # 5. Trim và chuẩn hoá khoảng trắng
    text = re.sub(r'\r', '\n', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return text.strip()

def _extract_from_dict_format(text: str) -> dict:
    """
    Trích dạng dict từ output của model.
    Ưu tiên dict có đủ 2–4 mục hợp lệ. Không crash, không mất dữ liệu.
    """
    sections = {}
    if not text:
        return sections

    # Tìm tất cả block {...}
    dict_matches = re.findall(r'\{[^{}]{10,2000}\}',text)

    if not dict_matches:
        return {}

    best = None
    best_score = -1

    # Chọn dict có nhiều key hợp lệ nhất
    for d in dict_matches:
        test = d.replace("“", '"').replace("”", '"').replace("'", '"')
        try:
            parsed = json.loads(test)
            if not isinstance(parsed, dict):
               continue

            valid_keys = ["lĩnh", "linh", "căn", "can", "phân", "phan", "kết", "ket"]
            if len(parsed) > 6:
               continue
            if not any(any(vk in k.lower() for vk in valid_keys) for k in parsed.keys()):
               continue
            keys = parsed.keys()
            score = sum(k in keys for k in ['Lĩnh vực', 'Căn cứ pháp lý', 'Phân tích', 'Kết luận'])
            if score > best_score:
                best_score = score
                best = parsed
        except Exception:
            continue

    if not best:
        return {}

    parsed = best

    # Hàm lấy giá trị theo nhiều tên khả dĩ
    def take(*keys):
        for k, v in parsed.items():
            if any(x.lower() in k.lower() for x in keys):
                return str(v).strip()
        return ""

    val = take("lĩnh", "linh", "field", "area")
    if isinstance(val, dict):
        val = ""
    sections = {
    "linh_vuc": val,
    "can_cu": take("căn", "can", "legal", "basis"),
    "phan_tich": take("phân", "phan", "analysis", "reasoning"),
    "ket_luan": take("kết", "ket", "conclusion", "result")
}


    # Chỉ trả các key có nội dung thật sự
    sections = {k: v for k, v in sections.items() if v and len(v) > 3}

    return sections

    
def _parse_four_sections(text: str) -> dict:
    """
    Parse văn bản thành 4 phần bắt buộc.
    Trả về dict với keys: linh_vuc, can_cu, phan_tich, ket_luan
    """
    text = _clean_model_output(text)
    sections = {"linh_vuc": "", "can_cu": "", "phan_tich": "", "ket_luan": ""}

    # ===== FIX: tách block dict JSON-like trước khi regex =====
    dict_sections = _extract_from_dict_format(text)
    if dict_sections:
        return dict_sections

    # Nếu không parse được dict → khi đó mới loại bỏ block dict để regex không ăn nhầm
    m_dict = re.search(r'\{[\s\S]{10,5000}\}', text)
    if m_dict:
        text = text.replace(m_dict.group(0), "").strip()

    patterns = {
    "linh_vuc": r"(?:^|\n)\s*Lĩnh vực\s*[:\-–]\s*(.+?)(?=\n\S)",
    "can_cu": r"(?:^|\n)\s*Căn cứ pháp lý\s*[:\-–]\s*(.+?)(?=\n\S)",
    "phan_tich": r"(?:^|\n)\s*Phân tích\s*[:\-–]\s*(.+?)(?=\n\S)",
    "ket_luan": r"(?:^|\n)\s*Kết luận\s*[:\-–]\s*(.+)$"
    }
    for key, patt in patterns.items():
        m = re.search(patt, text, re.DOTALL)
        if m:
           sections[key] = m.group(1).strip()

    # Nếu vẫn chưa có, thử tách theo đoạn văn (double newline) và map heuristically
    if not any(sections.values()):
        parts = [p.strip() for p in re.split(r'\n\n+', text) if p.strip()]
        if parts:
            # heuristic mapping: đầu tiên -> lĩnh vực, tiếp -> căn cứ, tiếp -> phân tích, cuối -> kết luận
            order = ['linh_vuc', 'can_cu', 'phan_tich', 'ket_luan']
            for i, part in enumerate(parts[:4]):
                sections[order[i]] = part

    return sections


def _deduplicate_sentences(text: str, threshold: float = 0.82) -> str:
    """
    Loại câu lặp dựa trên cosine embedding trong từng phần.
    Giữ lại câu mang thông tin mới.
    """
    if not text:
        return ""

    text = text.strip()
    parts = re.split(r'\n\s*\n', text)
    cleaned_parts = []

    for part in parts:
        sents = re.split(r'(?<=[.?!])\s+', part)
        sents = [s.strip() for s in sents if s.strip()]
        if len(sents) <= 1:
            cleaned_parts.append(part.strip())
            continue

        try:
            embs = _embed_sentences_cached(sents)
        except:
            cleaned_parts.append(part.strip())
            continue

        kept = [sents[0]]
        kept_embs = [embs[0]]

        for i in range(1, len(sents)):
            score = util.cos_sim(embs[i], torch.stack(kept_embs)).max().item()
            if score < threshold:
                kept.append(sents[i])
                kept_embs.append(embs[i])

        cleaned_parts.append(" ".join(kept).strip())

    return "\n\n".join(cleaned_parts).strip()



def extract_legal_conclusion(text: str):
    """Trích xuất kết luận pháp lý từ văn bản"""
    if not text:
        return ""
    m = re.search(
        r'([^.?!]*?(Điều\s*\d+|Khoản\s*\d+|Quyết\s*định|Kết\s*luận)[^.?!]*[.?!])',
        text, re.I
    )
    if m:
        return m.group(1).strip()
    sents = re.split(r'(?<=[.?!])\s+', text.strip())
    for s in reversed(sents[-3:]):
        if len(s.strip()) > 10:
            return s.strip()
    return text.strip()


def _validate_and_complete_sections(sections: dict, original_text: str) -> dict:
    """
    Kiểm tra và bổ sung các phần còn thiếu.
    Cải tiến: phát hiện lĩnh vực pháp lý linh hoạt, tránh mặc định 'Dân sự' cho mọi trường hợp.
    """
    cleaned_original = _clean_model_output(original_text)
    lower_text = cleaned_original.lower()

    # Đảm bảo đủ khóa
    for k in ['linh_vuc', 'can_cu', 'phan_tich', 'ket_luan']:
        sections.setdefault(k, "")

    # === 🔹 Phát hiện lĩnh vực pháp lý ===
    if not sections['linh_vuc']:
        field_map = {
            "dân sự": [
                "hợp đồng", "bồi thường", "di chúc", "thừa kế", "quyền sở hữu", "tài sản", "giao dịch dân sự"
            ],
            "hôn nhân và gia đình": [
                "ly hôn", "kết hôn", "nuôi con", "tài sản chung", "tài sản riêng", "hôn nhân", "gia đình", "cha mẹ", "con cái"
            ],
            "lao động": [
                "người lao động", "hợp đồng lao động", "tiền lương", "nghỉ việc", "sa thải", "bảo hiểm xã hội", "làm việc", "trợ cấp"
            ]
        }

        scores = Counter()
        for field, keywords in field_map.items():
            for kw in keywords:
                if re.search(rf"\b{re.escape(kw)}\b", lower_text):
                    scores[field] += 1

        if scores:
            best_field = scores.most_common(1)[0][0]
            sections['linh_vuc'] = f"Luật {best_field.title()}"
        else:
            sections['linh_vuc'] = "Chưa xác định rõ – cần căn cứ thêm vào tình tiết"

    # === 🔹 Bổ sung căn cứ pháp lý ===
    if not sections['can_cu']:
        legal_refs = re.findall(r'(Điều\s+\d+[a-zA-Z]?|Khoản\s+\d+|Chương\s+[IVXLC]+)', cleaned_original, re.IGNORECASE)
        if legal_refs:
            seen = set()
            unique_refs = []
            for ref in legal_refs:
                if ref not in seen:
                    seen.add(ref)
                    unique_refs.append(ref)
            sections['can_cu'] = ", ".join(unique_refs[:5])
        else:
            sections['can_cu'] = "Theo quy định của pháp luật hiện hành"

    # === 🔹 Bổ sung phần phân tích ===
    if not sections['phan_tich']:
        sentences = re.split(r'(?<=[.?!])\s+', cleaned_original.strip())
        meaningful = [s for s in sentences if len(s) > 30 and not s.strip().startswith(('1.', '2.', '3.', '4.', '###'))]
        if meaningful:
            sections['phan_tich'] = " ".join(meaningful[:3])
        else:
            sections['phan_tich'] = "Theo quy định pháp luật, cần xem xét các yếu tố liên quan."

    # === 🔹 Bổ sung phần kết luận ===
    if not sections['ket_luan'] or len(sections['ket_luan']) < 15:
        concl = extract_legal_conclusion(cleaned_original)
        if concl:
            sections['ket_luan'] = concl
        else:
            sections['ket_luan'] = "Cần xem xét cụ thể từng trường hợp theo quy định pháp luật."

    # === 🔹 Chuẩn hoá lại các phần ===
    for key in sections:
        if sections[key]:
            sections[key] = re.sub(r'^[\-–—\s]+', '', sections[key]).strip()

    return sections


def _legal_style_cleanup(text: str) -> str:
    """
    Chuẩn hóa văn phong pháp lý: khách quan, chính xác, bỏ cảm tính.
    """
    if not text:
        return text

    text = re.sub(r'\b(rõ ràng|hiển nhiên|theo quan điểm của tôi|chúng ta thấy rằng|tôi cho rằng)\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'điều\s+(\d+)', lambda m: f"Điều {m.group(1)}", text, flags=re.IGNORECASE)
    text = text.replace(" - ", " – ")
    text = text.strip()
    return text


def _format_legal_answer_enhanced(answer: str) -> str:
    """
    Chuẩn hoá câu trả lời thành 4 phần: Lĩnh vực, Căn cứ pháp lý, Phân tích, Kết luận
    ĐÃ FIX lỗi trộn nhầm Căn cứ vào Lĩnh vực và trùng đánh số.
    """
    text = _clean_model_output(answer)
    sections = _parse_four_sections(text)
    sections = _validate_and_complete_sections(sections, text)
    if all(sections[k] for k in ['linh_vuc', 'can_cu', 'phan_tich', 'ket_luan']):
        return (
        f"1. **Lĩnh vực** – {sections['linh_vuc']}\n\n"
        f"2. **Căn cứ pháp lý** – {sections['can_cu']}\n\n"
        f"3. **Phân tích** – {sections['phan_tich']}\n\n"
        f"4. **Kết luận** – {sections['ket_luan']}"
    ).strip()
    # Loại bỏ numbering/bold dư ở đầu phần
    for key in sections:
        if sections[key]:
            sections[key] = re.sub(
                r'^\s*\d+\s*[.)\-–]*\s*', '',  # xóa "1. ", "1)", "1 -", ...
                sections[key], flags=re.IGNORECASE).strip()
            sections[key] = re.sub(
                r'^\*\*.+?\*\*\s*[–:\-]\s*', '',
                sections[key], flags=re.IGNORECASE).strip()

    # Dọn văn phong
    for k in sections:
        sections[k] = _legal_style_cleanup(_deduplicate_sentences(sections[k]))

    # Construct final standardized format (Duy nhất 1 lần numbering)
    return (
        f"1. **Lĩnh vực** – {sections['linh_vuc']}\n\n"
        f"2. **Căn cứ pháp lý** – {sections['can_cu']}\n\n"
        f"3. **Phân tích** – {sections['phan_tich']}\n\n"
        f"4. **Kết luận** – {sections['ket_luan']}"
    ).strip()


# === Main RAG orchestration (example simplified) ===
# Note: The functions below assume `retriever` and `decoder` objects are provided

def _retrieve_with_cache(retriever, query, top_k=5, use_cache=True):
    key = (query, top_k)
    if use_cache and key in _RETRIEVAL_CACHE:
        return _RETRIEVAL_CACHE[key]
    retrieved = retriever.retrieve(query, top_k=top_k)
    normalized = []
    for item in retrieved:
        try:
            if isinstance(item, (list, tuple)) and len(item) >= 1:
                normalized.append((item[0], item[1] if len(item) > 1 else None))
            else:
                normalized.append((str(item), None))
        except Exception:
            normalized.append((str(item), None))
    if use_cache:
        _RETRIEVAL_CACHE[key] = normalized
    return normalized


def legal_self_consistency_rag(
    retriever,
    decoder,
    query: str,
    num_samples: int = 3,
    temperature: float = 0.5,
    max_new_tokens: int = 384,
    top_k: int = 3,
    use_retrieval_cache: bool = True,
    dedup_threshold: float = 0.85,
    enforce_structure: bool = True
):
    """
    RAG với self-consistency: sinh nhiều hướng lập luận, chuẩn hoá sau khi chọn kết quả tốt nhất.
    """
    t0 = time.perf_counter()
    retrieved = _retrieve_with_cache(retriever, query, top_k=top_k, use_cache=use_retrieval_cache)
    context_text = "\n\n".join([doc for doc, _ in retrieved])
    prompt = decoder._build_context(query, context_text)

    # 🔹 outputs_raw = chứa các câu trả lời CHƯA format
    outputs_raw = []
    print("\n🔄 Đang sinh các hướng lập luận:")

    for i in range(num_samples):
        try:
            out = decoder.decode(prompt, temperature=temperature, max_new_tokens=max_new_tokens, num_samples=1)
            text_out = out[0].strip() if isinstance(out, (list, tuple)) else str(out).strip()
            if not text_out:
                print(f"⚠️ Hướng {i+1}: Output rỗng, bỏ qua")
                continue

            text_out = _clean_model_output(text_out)  # chỉ clean, KHÔNG format
            outputs_raw.append(text_out)
            print(f"✅ Hướng {i+1} hoàn thành ({len(text_out)} ký tự)")

        except Exception as e:
            print(f"⚠️ Lỗi khi sinh hướng {i+1}: {e}")

    if not outputs_raw:
        return {"final_answer": "Không sinh được câu trả lời hợp lệ.", "candidates": []}

    # === Self-consistency: chọn câu trả lời trung bình về ngữ nghĩa ===
    print("\n🧠 Đang đánh giá độ nhất quán ngữ nghĩa...")
    embs = _embed_sentences_cached(outputs_raw)
    sims = util.cos_sim(embs, embs)
    avg_scores = sims.mean(dim=1)
    best_idx = int(torch.argmax(avg_scores))

    # 🔥 Format CHỈ ứng viên tốt nhất
    final_answer = (
        _format_legal_answer_enhanced(outputs_raw[best_idx])
        if enforce_structure else outputs_raw[best_idx]
    )

    # === Format các candidates còn lại (tuỳ chọn)
    candidates = [
        _format_legal_answer_enhanced(c) if enforce_structure else c
        for c in outputs_raw
    ]

    elapsed = time.perf_counter() - t0
    print(f"\n⏱️ Hoàn tất RAG trong {elapsed:.2f}s — chọn hướng {best_idx + 1}")

    return {"final_answer": final_answer, "candidates": candidates, "best_idx": best_idx}



In [16]:
decoder = Decoder('/kaggle/input/llama_law_instruct_v3/keras/default/1')
question = 'Chị M 16 tuổi và anh R 20 tuổi yêu nhau và muốn làm thủ tục kết hôn. Pháp luật có cho phép không?'
result = legal_self_consistency_rag(retriever, decoder, question, num_samples = 2)
print('Câu trả lời cuối cùng:')
print(result["final_answer"])

The tokenizer you are loading from '/kaggle/input/llama_law_instruct_v3/keras/default/1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔎 Truy vấn: Chị M 16 tuổi và anh R 20 tuổi yêu nhau và muốn làm thủ tục kết hôn. Pháp luật có cho phép không?
📑 Top 3 kết quả:
   01. (0.9782) Điều 8. Điều kiện kết hôn [KHOẢN] 1. Nam, nữ kết hôn với nhau phải tuân theo các điều kiện sau đây: a) Nam từ đủ 20 tuổi trở lên, nữ từ đủ 18 tuổi trở lên; b) Việc kết hôn do nam và nữ tự nguyện quyết...
   02. (0.1977) [CHƯƠNG] CHƯƠNG III QUAN HỆ GIỮA VỢ VÀ CHỒNG [MỤC] Mục 1 QUYỀN VÀ NGHĨA VỤ VỀ NHÂN THÂN [ Điều 16. Giải quyết quan hệ tài sản, nghĩa vụ và hợp đồng của nam, nữ chung sống với nhau như vợ chồng mà khôn...
   03. (0.0745) Điều 8 của Luật này. [KHOẢN] 9. Cưỡng ép kết hôn, ly hôn là việc đe dọa, uy hiếp tinh thần, hành hạ, ngược đãi, yêu sách của cải hoặc hành vi khác để buộc người khác phải kết hôn hoặc ly hôn trái với ...

🔄 Đang sinh các hướng lập luận:

🧠 Hướng #1 (độ dài: 361 ký tự)
{'Căn cứ pháp lý': 'Điểm b Khoản 1 Điều 8', 'Kết luận': 'Chị M và anh R có thể kết hôn vì họ đủ điều kiện về độ tuổi và không thuộc trường hợp cấm k

In [17]:
# === Thêm import cần thiết cho metrics mới ===
import nltk
nltk.download('punkt')  # chỉ chạy 1 lần nếu chưa có
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
import re

# ===============================
# 🔧 Hàm chuẩn hóa và tokenize
# ===============================
def normalize_answer_for_metrics(ans):
    """
    Chuyển answer (có thể là dict hoặc str) -> text thuần để tính BLEU/METEOR/ROUGE.
    Nếu ans là dict (ví dụ 4 mục), nối theo key order để giữ cấu trúc.
    """
    if isinstance(ans, dict):
        # giữ thứ tự xuất hiện trong dict
        parts = []
        for k, v in ans.items():
            parts.append(f"{k}: {v}")
        text = " ".join(parts)
    else:
        text = str(ans)

    # Loại các markdown **, ký tự đặc biệt nhiều dòng, rút gọn khoảng trắng
    text = re.sub(r"\*\*", "", text)
    text = re.sub(r"\r\n|\r|\n", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def tokenize_for_meteor(text):
    """
    METEOR của NLTK cần tokenized input (list of tokens).
    Dùng nltk.word_tokenize để token hóa tiếng (đã download punkt).
    """
    return nltk.word_tokenize(text)

# ===============================
# 📊 Hàm tính BLEU / METEOR / ROUGE
# ===============================
def evaluate_bleu_meteor_rouge(references, predictions):
    """
    Input:
      - references: list[str] (đã normalize)
      - predictions: list[str] (đã normalize)
    Output:
      - detailed: list[dict] per-sample metrics
      - summary: dict of average metrics
    """
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    smooth_fn = SmoothingFunction().method1

    detailed = []
    bleu_list, meteor_list, rouge1_list, rouge2_list, rougeL_list = [], [], [], [], []

    for ref, pred in zip(references, predictions):
        # tokenized lists
        ref_tokens = tokenize_for_meteor(ref)
        pred_tokens = tokenize_for_meteor(pred)

        # BLEU (sentence-level) on token lists
        try:
            bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth_fn)
        except Exception:
            bleu = 0.0

        # METEOR expects tokenized iterables
        try:
            meteor = meteor_score([ref_tokens], pred_tokens)
        except Exception:
            meteor = 0.0

        # ROUGE on raw text
        r = scorer.score(ref, pred)
        rouge1 = r["rouge1"].fmeasure
        rouge2 = r["rouge2"].fmeasure
        rougeL = r["rougeL"].fmeasure

        bleu_list.append(bleu)
        meteor_list.append(meteor)
        rouge1_list.append(rouge1)
        rouge2_list.append(rouge2)
        rougeL_list.append(rougeL)

        detailed.append({
            "BLEU": float(bleu),
            "METEOR": float(meteor),
            "ROUGE": {
                "ROUGE-1": float(rouge1),
                "ROUGE-2": float(rouge2),
                "ROUGE-L": float(rougeL),
            }
        })

    n = max(1, len(detailed))
    summary = {
        "Avg BLEU": sum(bleu_list) / n,
        "Avg METEOR": sum(meteor_list) / n,
        "Avg ROUGE-1": sum(rouge1_list) / n,
        "Avg ROUGE-2": sum(rouge2_list) / n,
        "Avg ROUGE-L": sum(rougeL_list) / n,
    }

    return detailed, summary

# ===============================
# 🔁 Run RAG Self-Consistency Pipeline (đã chỉnh sử dụng BLEU/METEOR/ROUGE)
# ===============================
def run_rag_selfconsistency_pipeline(
    json_folder,
    retriever,
    decoder,
    output_dir="results_json",
    num_samples=3,
    temperature=0.5,
    max_new_tokens=384,
    top_k=3,
    use_retrieval_cache=True,
    dedup_threshold=0.85
):
    if "legal_self_consistency_rag" not in globals():
        raise RuntimeError("❌ Hàm legal_self_consistency_rag chưa được import!")

    os.makedirs(output_dir, exist_ok=True)
    all_refs, all_preds, results = [], [], []
    json_files = [f for f in os.listdir(json_folder) if f.endswith(".json")]

    print(f"\n🚀 Bắt đầu đánh giá (num_samples={num_samples}) — {len(json_files)} files\n")

    for filename in tqdm(json_files, desc="⚖️ Xử lý các file"):
        filepath = os.path.join(json_folder, filename)
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print(f"❌ Lỗi đọc {filename}: {e}")
            continue

        entries = data if isinstance(data, list) else [data]

        for idx, item in enumerate(entries):
            question = item.get("question", "")
            reference = item.get("answer", "")
            context = item.get("context", "")
            pdf_file = item.get("file", "")

            if not question:
                print(f"⚠️ Bỏ qua item {idx} — thiếu question")
                continue

            try:
                print("\n" + "=" * 80)
                print(f"📄 {filename} — Câu {idx+1}/{len(entries)}")
                print(f"📘 {question[:200]}...")

                start = time.time()
                result = legal_self_consistency_rag(
                    retriever=retriever,
                    decoder=decoder,
                    query=question,
                    num_samples=num_samples,
                    temperature=temperature,
                    max_new_tokens=max_new_tokens,
                    top_k=top_k,
                    use_retrieval_cache=use_retrieval_cache,
                    dedup_threshold=dedup_threshold
                )
                elapsed = round(time.time() - start, 2)

                all_candidates = result.get("candidates", [])
                print(f"\n Các hướng suy nghĩ cụ thể: ", all_candidates)

                if not all_candidates:
                    print(f"⚠️ Không có candidate hợp lệ cho câu {idx}")
                    continue

                best_idx = result.get("best_idx", 0)
                # bảo đảm index hợp lệ
                if not isinstance(best_idx, int) or best_idx < 0 or best_idx >= len(all_candidates):
                    best_idx = 0

                best_answer = all_candidates[best_idx]
                print(f"\n💬 Câu trả lời cuối cùng:")
                print("-" * 70)
                print(best_answer)
                print("-" * 70)

                # ====== NORMALIZE trước khi lưu vào all_refs/all_preds ======
                norm_ref = normalize_answer_for_metrics(reference)
                norm_pred = normalize_answer_for_metrics(best_answer)

                all_refs.append(norm_ref)
                all_preds.append(norm_pred)

                results.append({
                    "question": question,
                    "reference_answer": reference,
                    "model_answer": best_answer,
                    "candidates": all_candidates,
                    "num_samples": len(all_candidates),
                    "context": context,
                    "pdf_file": pdf_file,
                    "timing_seconds": elapsed,
                })

                # Lưu file riêng
                base = os.path.splitext(filename)[0]
                out_path = os.path.join(output_dir, f"{base}_q{idx+1}_result.json")
                with open(out_path, "w", encoding="utf-8") as fw:
                    json.dump(results[-1], fw, ensure_ascii=False, indent=2)

            except Exception as e:
                print(f"❌ Lỗi xử lý {filename}, câu {idx}: {repr(e)}")
                import traceback; traceback.print_exc()
                continue

        torch.cuda.empty_cache()
        gc.collect()

    if not all_refs:
        print("\n⚠️ Không có kết quả để đánh giá.")
        return [], []

    print("\n📊 Đang tính metrics tổng thể (BLEU/METEOR/ROUGE)...")
    detailed, summary = evaluate_bleu_meteor_rouge(all_refs, all_preds)

    for i, r in enumerate(results):
        if i < len(detailed):
            r["metrics"] = detailed[i]

    avg_time = sum(r.get("timing_seconds", 0) for r in results) / max(1, len(results))
    summary["Avg_Generation_Time_s"] = round(avg_time, 2)
    summary["Total_Questions"] = len(results)

    final_output = {"results": results, "overall_metrics": summary}
    with open(os.path.join(output_dir, "rag_selfconsistency_results.json"), "w", encoding="utf-8") as f:
        json.dump(final_output, f, ensure_ascii=False, indent=2)

    print("\n✅ Đã lưu kết quả tổng hợp.")
    return all_preds, results


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
json_folder = '/kaggle/input/law-benchmark/Benchmark'
predictions, results = run_rag_selfconsistency_pipeline(json_folder, retriever, decoder, output_dir="rag_selfconsistency_pipeline_results.json")


🚀 Bắt đầu đánh giá (num_samples=3) — 9 files



⚖️ Xử lý các file:   0%|          | 0/9 [00:00<?, ?it/s]


📄 71-80.json — Câu 1/10
📘 Anh D (chưa thành niên) được cha mẹ cho phép sử dụng một tài khoản chứng khoán đứng tên anh. Anh D muốn tự mình thực hiện giao dịch liên quan đến tài khoản này. Việc này có được không?...
🔎 Truy vấn: Anh D (chưa thành niên) được cha mẹ cho phép sử dụng một tài khoản chứng khoán đứng tên anh. Anh D muốn tự mình thực hiện giao dịch liên quan đến tài khoản này. Việc này có được không?
📑 Top 3 kết quả:
   01. (0.9960) Điều 85. Hạn chế quyền của cha, mẹ đối với con chưa thành niên [KHOẢN] 1. Cha, mẹ bị hạn chế quyền đối với con chưa thành niên trong các trường hợp sau đây: a) Bị kết án về một trong các tội xâm phạm ...
   02. (0.9949) Điều 73. Đại diện cho con [KHOẢN] 1. Cha mẹ là người đại diện theo pháp luật của con chưa thành niên, con đã thành niên mất năng lực hành vi dân sự, trừ trường hợp con có người khác làm giám hộ hoặc c...
   03. (0.9901) Điều 112. Nghĩa vụ cấp dưỡng giữa anh, chị, em Trong trường hợp không còn cha mẹ hoặc cha mẹ không có khả năng lao 

⚖️ Xử lý các file:  11%|█         | 1/9 [05:11<41:33, 311.72s/it]


📄 81-90.json — Câu 1/10
📘 Anh X và chị Y sống chung với nhau trong thời kỳ hôn nhân. Thu nhập do lao động của anh X tạo ra là 30 triệu/tháng. Theo Luật, khoản thu nhập này là tài sản chung hay tài sản riêng của anh X?...
🔎 Truy vấn: Anh X và chị Y sống chung với nhau trong thời kỳ hôn nhân. Thu nhập do lao động của anh X tạo ra là 30 triệu/tháng. Theo Luật, khoản thu nhập này là tài sản chung hay tài sản riêng của anh X?
📑 Top 3 kết quả:
   01. (0.9932) [CHƯƠNG] CHƯƠNG III QUAN HỆ GIỮA VỢ VÀ CHỒNG [MỤC] Mục 1 QUYỀN VÀ NGHĨA VỤ VỀ NHÂN THÂN [ Điều 16. Giải quyết quan hệ tài sản, nghĩa vụ và hợp đồng của nam, nữ chung sống với nhau như vợ chồng mà khôn...
   02. (0.9835) Điều 33. Tài sản chung của vợ chồng [KHOẢN] 1. Tài sản chung của vợ chồng gồm tài sản do vợ, chồng tạo ra, thu nhập do lao động, hoạt động sản xuất, kinh doanh, hoa lợi, lợi tức phát sinh từ tài sản r...
   03. (0.9520) Điều 112. Nghĩa vụ cấp dưỡng giữa anh, chị, em Trong trường hợp không còn cha mẹ hoặc cha mẹ không có

...


📄 51-60.json — Câu 1/10
📘 Công ty X ký hợp đồng lao động xác định thời hạn 36 tháng với anh B. Sau khi hợp đồng hết hạn 36 tháng, anh B vẫn tiếp tục làm việc và sau 40 ngày kể từ ngày hợp đồng hết hạn, hai bên vẫn chưa ký hợp ...
🔎 Truy vấn: Công ty X ký hợp đồng lao động xác định thời hạn 36 tháng với anh B. Sau khi hợp đồng hết hạn 36 tháng, anh B vẫn tiếp tục làm việc và sau 40 ngày kể từ ngày hợp đồng hết hạn, hai bên vẫn chưa ký hợp đồng lao động mới. Loại hợp đồng lao động tiếp theo sẽ là gì?
📑 Top 3 kết quả:
   01. (0.9922) Điều 36. Quyền đơn phương chấm đứt hợp đồng lao động của người sử dụng lao động [KHOẢN] 1. Người sử dụng lao động có quyền đơn phương chấm dứt hợp đồng lao động trong trường hợp sau đây: a) Người lao ...
   02. (0.9877) Điều 83. Thỏa ước lao động tập thể hết hạn Trong thời hạn 20 ngày trước ngày thỏa ước lao động tập thể hết hạn, các bên có thể thương lượng để kéo đài thời hạn của thỏa ước lao động tập thể hoặc ký kế...
   03. (0.9838) Điều 162. Hợp đồng lao

⚖️ Xử lý các file:  89%|████████▉ | 8/9 [43:53<05:14, 314.92s/it]


📄 21-30.json — Câu 1/10
📘 Các loại Đại diện của Pháp nhân theo pháp luật là ai?...
🔎 Truy vấn: Các loại Đại diện của Pháp nhân theo pháp luật là ai?
📑 Top 3 kết quả:
   01. (0.9993) Điều 85. Đại diện của pháp nhân Đại diện của pháp nhân có thể là đại diện theo pháp luật hoặc đại diện theo ủy quyền. Người đại diện của pháp nhân phải tuân theo quy định về đại diện tại Chương IX Phầ...
   02. (0.9991) Điều 134. Đại diện [KHOẢN] 1. Đại diện là việc cá nhân, pháp nhân (sau đây gọi chung là người đại diện) nhân danh và vì lợi ích của cá nhân hoặc pháp nhân khác (sau đây gọi chung là người được đại diệ...
   03. (0.9980) Điều 137. Đại diện theo pháp luật của pháp nhân [KHOẢN] 1. Người đại diện theo pháp luật của pháp nhân bao gồm: a) Người được pháp nhân chỉ định theo điều lệ; b) Người có thẩm quyền đại diện theo quy ...

🔄 Đang sinh các hướng lập luận:

🧠 Hướng #1 (độ dài: 1306 ký tự)
{'Căn cứ pháp lý': 'Khoản 1 Điều 85, Khoản 1 Điều 137', 'Kết luận': 'Các loại đại diện của pháp nhân bao gồ

⚖️ Xử lý các file: 100%|██████████| 9/9 [50:12<00:00, 334.76s/it]



📊 Đang tính metrics tổng thể (BLEU/METEOR/ROUGE)...

✅ Đã lưu kết quả tổng hợp.


In [19]:
!zip -r results_selfconsistency_llama_json.zip /kaggle/working/rag_selfconsistency_pipeline_results.json

  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/ (stored 0%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/41-50_q9_result.json (deflated 85%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/31-40_q1_result.json (deflated 83%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/51-60_q7_result.json (deflated 77%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/rag_selfconsistency_results.json (deflated 89%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/61-70_q6_result.json (deflated 81%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/21-30_q1_result.json (deflated 84%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/61-70_q7_result.json (deflated 74%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/51-60_q3_result.json (deflated 76%)
  adding: kaggle/working/rag_selfconsistency_pipeline_results.json/61-70_q10_result.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
